In [2]:
#!/usr/bin/env python
# coding: utf-8
import numpy
import sklearn
import dateutil
import sqlite3
from sqlite3 import Error
from sklearn import preprocessing
import pandas as pd
import pickle
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import platform
import os
from tpot import TPOTRegressor
from sklearn.model_selection import train_test_split

In [3]:
def create_connection(db_file):
        """ create a database connection to the SQLite database
            specified by the db_file
        :param db_file: database file
        :return: Connection object or None
        """
        try:
            conn = sqlite3.connect(db_file)
            return conn
        except Error as e:
            print(e)
        return None

In [4]:
Sql_connection1 = create_connection('/opt/apps/scripts/jaarvis_demand_supply/evo.db')

In [5]:
train_df = pd.read_sql_query("select * from zones order by id;", Sql_connection1)

In [6]:
train_df.booked_vehicles=train_df.booked_vehicles.fillna(0)
train_df.free_vehicles=train_df.free_vehicles.fillna(0)
train_df=train_df.dropna(how='any')

In [7]:
train_df = train_df[~train_df.astype(str).eq('None').any(1)]

In [8]:
train_df = train_df.mask(train_df.eq('None')).dropna()

In [9]:
train_df=train_df[~train_df.zone.apply(lambda x: str(x).isdigit())]

In [10]:
train_df.time = train_df.time.astype(str).str[:2].astype(int)

In [11]:
train_df.time = train_df.time.replace(regex=True, to_replace=r'\D', value=r'')

In [12]:
train_df=train_df.sort_values(['zone','year','month','week','day','time'],ascending=True)

In [13]:
train_df['date']=train_df[['year','month','day']].apply(lambda x: '-'.join(x.dropna().astype(str).values), axis=1)

In [14]:
train_df = train_df.drop(['id','year','month','week','day'],axis=1)

In [15]:
train_df['date']=train_df[['date','time']].apply(lambda x: ' '.join(x.dropna().astype(str).values), axis=1)

In [16]:
train_df['date']=train_df[['date','time']].apply(lambda x: x.astype(str)+':00:00', axis=1)

In [17]:
train_df=train_df.sort_values(['date','zone'])

In [18]:
# pivot_table(df, values='D', index=['A', 'B'],columns=['C'], aggfunc=np.sum)
train_booked_df=pd.pivot_table(train_df,values=['booked_vehicles'],index=['date'],columns=['zone'])
train_free_df=pd.pivot_table(train_df,values=['free_vehicles'],index=['date'],columns=['zone'])

In [19]:
train_booked_df = train_booked_df.fillna(0)
train_free_df = train_free_df.fillna(0)

In [20]:
train_booked_df=train_booked_df.booked_vehicles
train_free_df=train_free_df.free_vehicles

In [21]:
booked_df = train_booked_df.reindex(pd.date_range(start='2018-11-16 14:00:00', end=train_booked_df.index[-1], freq='H'))
free_df = train_free_df.reindex(pd.date_range(start='2018-11-16 14:00:00', end=train_free_df.index[-1], freq='H'))

In [22]:
booked_df = booked_df[~booked_df.index.isin(train_booked_df.index)]
free_df = free_df[~free_df.index.isin(train_free_df.index)]

In [23]:
train_booked_df=pd.concat([train_booked_df,booked_df])
train_free_df = pd.concat([train_free_df,free_df])

In [24]:
import datetime
train_booked_df=train_booked_df.iloc[pd.to_datetime(train_booked_df.index).values.argsort()]
train_free_df=train_free_df.iloc[pd.to_datetime(train_free_df.index).values.argsort()]
# train_booked_df.index = pd.to_datetime(train_booked_df.index)
# train_free_df.index = pd.to_datetime(train_free_df.index)
train_booked_df = train_booked_df.fillna(0)
train_free_df = train_free_df.fillna(0)
# train_booked_df.sort_index()
# train_free_df.sort_index()

In [25]:
train_booked_df.index = pd.to_datetime(train_booked_df.index) - pd.Timedelta(hours=5)
train_free_df.index = pd.to_datetime(train_free_df.index) - pd.Timedelta(hours=5)

In [26]:
import statsmodels.api
# from pyramid.arima import auto_arima
# model = auto_arima(train_booked_df['V5M'][48+168*0:48+168*1], trace=True, error_action='ignore', suppress_warnings=True)
# model.fit(train_booked_df['V5M'][48+168*0:48+168*1])
# forecast = model.predict(n_periods=24)
# forecast

In [27]:
model2=statsmodels.api.tsa.SARIMAX(train_booked_df.V5R[-171:-171+168],order=(1,0,1),seasonal_order=(1,1,0,24))
model2 = model2.fit()

/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)


LinAlgError: Non-positive-definite forecast error covariance matrix encountered at period 24

In [ ]:
# import matplotlib.pyplot as plt
# %matplotlib inline 
# import statsmodels
# plt.rcParams['figure.figsize'] = [20,10]
# plt.tight_layout()
# fig,ax = plt.subplots()
# fig1 = ax.plot(train_booked_df.V5R[-171:-171+168].values, label='actual' )
# fig2 = ax.plot(numpy.ceil(numpy.array(model2.forecast(24))), label= 'predicted' )
# ax.legend(fancybox=True, loc='upper right', shadow=True)
# ax.set_xlabel('hours in a week')
# ax.set_ylabel('no. of vehicles')
# ax.set_title('V7P 6h/Jan/2019 - 13th/Jan/2019')
# plt.show()

In [ ]:
import matplotlib.pyplot as plt
# %matplotlib inline 
# import statsmodels
# plt.rcParams['figure.figsize'] = [10,5]
# plt.tight_layout()
# fig,ax = plt.subplots()
# fig1 = ax.plot(train_booked_df.V5R[-171-].values, label='actual' )
# fig2 = ax.plot(numpy.ceil(numpy.array(model2.forecast(24))), label= 'predicted' )
# ax.legend(fancybox=True, loc='upper right', shadow=True)
# ax.set_xlabel('hours in a week')
# ax.set_ylabel('no. of vehicles')
# ax.set_title('V7P 6h/Jan/2019 - 13th/Jan/2019')
# plt.show()# 

In [28]:
def booking_model_saving(path1,path2):
    os.chdir(path1)
    for x in train_booked_df.keys():
        try:
            
            model = statsmodels.api.tsa.SARIMAX(train_booked_df[x][-171:-171+168],order=(1,0,1),seasonal_order=(1,1,0,24),enforce_stationarity=False)
            model = model.fit()
            model = model.save(str(x)+'.pkl')
        except:
            print( x, 'model not created')
    print('demand_models_created')
    
    os.chdir(path2)
    for y in train_free_df.keys():
        try:
            
            model = statsmodels.api.tsa.SARIMAX(train_free_df[y][-171:-171+168],order=(1,0,1),seasonal_order=(1,1,0,24),enforce_stationarity=False)
            model = model.fit()
            model = model.save(str(y)+'.pkl')
        except:
            print( y, 'model not created')
    print('available_models_created')

In [ ]:
booking_model_saving('/opt/apps/Jaarvis_Demand_Supply/demand_models','/opt/apps/Jaarvis_Demand_Supply/available_models')

/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)


V1M 2R6 model not created
V1M 3G1 model not created
V2J 3A9 model not created


/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)


V3J 3Z9 model not created
V3K 7B3 model not created
V3L 0A4 model not created
V3L 0A5 model not created


/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)


...

V6B 6L4 model not created


/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packa

V6B 6R5 model not created
V6C model not created
V6C 0B9 model not created
V6C 1A8 model not created
V6C 1E1 model not created
V6C 1G8 model not created
V6C 1N5 model not created
V6C 1S4 model not created
V6C 1T2 model not created
V6C 1V4 model not created


/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)


V6C 1X5 model not created
V6C 1X6 model not created
V6C 2L1 model not created


In [50]:
# %matplotlib inline 
# import statsmodels
# plt.rcParams['figure.figsize'] = [12,6]
# train_booked_df.V5R[-171:-171+168].plot()
# model2.forecast(24*3).plot()
# plt.legend(loc=4)
# plt.xlabel('hours')
# plt.ylabel('Price')
# plt.show()

In [96]:
import glob
import pickle
import json
from collections import defaultdict
def model_load_predict(path1,path2):
    booked=defaultdict(dict)
    available = defaultdict(dict)
    demand = {}
    for x in glob.glob(os.path.join(path1, '*.pkl')):
        try:
            model_book = pickle.load(open(x, 'rb'))
            zone= re.findall('[a-zA-Z]\d[a-zA-Z]\s{0,1}\d{0,1}[a-zA-Z]{0,1}\d{0,1}',x)[0]
            print(zone)
            booked[zone]['cars'] = [0 if i < 0 else i for i in numpy.ceil((model_book.forecast(24)))]
            print(lat_long_df[lat_long_df['zone'].str.contains(zone)]['lat'].values[0],lat_long_df[lat_long_df['zone'].str.contains(zone)]['log'].values[0])
            booked[zone]['lat_long'] = lat_long_df[lat_long_df['zone'].str.contains(zone)]['lat'].values[0],lat_long_df[lat_long_df['zone'].str.contains(zone)]['log'].values[0]
        except:
            print('problem with zone '+zone)
    for y in glob.glob(os.path.join(path2, '*.pkl')):
        try:
            model_free = pickle.load(open(y, 'rb'))
            zone = re.findall('[a-zA-Z]\d[a-zA-Z]\s{0,1}\d{0,1}[a-zA-Z]{0,1}\d{0,1}',y)[0]
            print(zone)
            available[zone]['cars'] = [0 if i < 0 else i for i in numpy.ceil((model_free.forecast(24)))]
            print(lat_long_df[lat_long_df['zone'].str.contains(zone)]['lat'].values[0],lat_long_df[lat_long_df['zone'].str.contains(zone)]['log'].values[0])
            available[zone]['lat_long'] = lat_long_df[lat_long_df['zone'].str.contains(zone)]['lat'].values[0],lat_long_df[lat_long_df['zone'].str.contains(zone)]['log'].values[0]
        except:
            print('problem with zone '+zone)
    demand['booked'] = booked
    demand['available'] = available
    return json.dumps(demand)

In [97]:
model_load_predict('/opt/apps/Jaarvis_Demand_Supply/demand_models/','/opt/apps/Jaarvis_Demand_Supply/available_models/')

V6B 2J2
problem with zone V6B 2J2
V5Z 2T1
problem with zone V5Z 2T1
V5N
49.2663951 -123.0699288
V6B 5A1
problem with zone V6B 5A1
V5V 3M5
problem with zone V5V 3M5
V6A 1G7
problem with zone V6A 1G7
V3M 0G8
problem with zone V3M 0G8
V6M 2G8
problem with zone V6M 2G8
V6Z 3G5
problem with zone V6Z 3G5
V6Z 1B1
problem with zone V6Z 1B1
V3L 5B3
problem with zone V3L 5B3
V6M 1M3
problem with zone V6M 1M3
V6A 1R8
problem with zone V6A 1R8
V5T 1J3
problem with zone V5T 1J3
V6A 1A4
problem with zone V6A 1A4
V5P
49.23866229999999 -123.0601234
V6E 1P1
problem with zone V6E 1P1
V3M 4K4
49.2116003 -122.9353234
V3M 2A5
problem with zone V3M 2A5
V5Y 0B2
problem with zone V5Y 0B2
V6B 1B7
problem with zone V6B 1B7
V6B 2T5
problem with zone V6B 2T5
V6B 1C9
problem with zone V6B 1C9
V6Z 1Y6
problem with zone V6Z 1Y6
V5P 4K6
problem with zone V5P 4K6
V3L 3C2
problem with zone V3L 3C2
V3L 3C5
problem with zone V3L 3C5
V5T 2W6
problem with zone V5T 2W6
V5P 4K7
problem with zone V5P 4K7
V5N 1X8
problem with 

V5T 2K4
problem with zone V5T 2K4
V6A 1E9
problem with zone V6A 1E9
V6H 2K5
problem with zone V6H 2K5
V7M 2K1
problem with zone V7M 2K1
V3L 4G6
49.2209088 -122.9070855
V3L 3H7
problem with zone V3L 3H7
V6E 1V1
49.2787634 -123.1360312
V5Y 2Z6
problem with zone V5Y 2Z6
V5R 5G9
problem with zone V5R 5G9
V6B 3P3
problem with zone V6B 3P3
V5Z 2G9
problem with zone V5Z 2G9
V7N 1X1
problem with zone V7N 1X1
V6M 2V5
problem with zone V6M 2V5
V6E 1J7
problem with zone V6E 1J7
V6B 1E1
problem with zone V6B 1E1
V5G 4J3
problem with zone V5G 4J3
V6Z
49.2755926 -123.1293344
V5N 4L1
problem with zone V5N 4L1
V6B 3L9
problem with zone V6B 3L9
V6C 1T2
problem with zone V6C 1T2
V5R
49.2495001 -123.048236
V6E 1N4
problem with zone V6E 1N4
V5R 3Z5
49.2370448 -123.0428101
V6R 4K5
problem with zone V6R 4K5
V3L 2K1
problem with zone V3L 2K1
V5Y 1H4
problem with zone V5Y 1H4
V5L 1S3
problem with zone V5L 1S3
V7B 0A1
problem with zone V7B 0A1
V6A 4H6
problem with zone V6A 4H6
V5G 4L7
problem with zone V5G 4L7

V3M 0J2
problem with zone V3M 0J2
V6J 3H6
problem with zone V6J 3H6
V6Z 2M4
problem with zone V6Z 2M4
V5L 2W2
problem with zone V5L 2W2
V5N 1A9
49.269823 -123.0684064
V5K 4Z6
problem with zone V5K 4Z6
V5Z 3N2
problem with zone V5Z 3N2
V6G 3A5
problem with zone V6G 3A5
V5V 5E9
problem with zone V5V 5E9
V5X 4K1
problem with zone V5X 4K1
V6A 1E3
problem with zone V6A 1E3
V3M 2M9
problem with zone V3M 2M9
V6B 2J2
problem with zone V6B 2J2
V5Z 2T1
problem with zone V5Z 2T1
V5N
49.2663951 -123.0699288
V6B 5A1
problem with zone V6B 5A1
V5V 3M5
problem with zone V5V 3M5
V6A 1G7
problem with zone V6A 1G7
V3M 0G8
problem with zone V3M 0G8
V6Z 3G5
problem with zone V6Z 3G5
V6Z 1B1
problem with zone V6Z 1B1
V3L 5B3
problem with zone V3L 5B3
V6M 1M3
problem with zone V6M 1M3
V6A 1R8
problem with zone V6A 1R8
V5T 1J3
problem with zone V5T 1J3
V6A 1A4
problem with zone V6A 1A4
V3M 1A5
problem with zone V3M 1A5
V5P
49.23866229999999 -123.0601234
V6A 1G4
problem with zone V6A 1G4
V6E 1P1
problem with z

V5T 2K4
problem with zone V5T 2K4
V6A 1E9
problem with zone V6A 1E9
V6H 2K5
problem with zone V6H 2K5
V7M 2K1
problem with zone V7M 2K1
V6E 1V1
49.2787634 -123.1360312
V5Y 2Z6
problem with zone V5Y 2Z6
V6E 1B5
problem with zone V6E 1B5
V5R 5G9
problem with zone V5R 5G9
V6B 3P3
problem with zone V6B 3P3
V5Z 2G9
problem with zone V5Z 2G9
V6B 5A7
problem with zone V6B 5A7
V7N 1X1
problem with zone V7N 1X1
V6M 2V5
problem with zone V6M 2V5
V6E 1J7
problem with zone V6E 1J7
V6B 1E1
problem with zone V6B 1E1
V5G 4J3
problem with zone V5G 4J3
V6Z
49.2755926 -123.1293344
V5N 4L1
problem with zone V5N 4L1
V6B 3L9
problem with zone V6B 3L9
V6C 1T2
problem with zone V6C 1T2
V5R
49.2495001 -123.048236
V6E 1N4
problem with zone V6E 1N4
V5R 3Z5
49.2370448 -123.0428101
V6R 4K5
problem with zone V6R 4K5
V3L 2K1
problem with zone V3L 2K1
V5Y 1H4
problem with zone V5Y 1H4
V5L 1S3
problem with zone V5L 1S3
V7B 0A1
problem with zone V7B 0A1
V6A 4H6
problem with zone V6A 4H6
V5G 4L7
problem with zone V5G 4

V6Z 2M4
problem with zone V6Z 2M4
V5L 2W2
problem with zone V5L 2W2
V5T 3K4
problem with zone V5T 3K4
V5N 1A9
49.269823 -123.0684064
V5K 4Z6
problem with zone V5K 4Z6
V5Z 3N2
problem with zone V5Z 3N2
V5V 5E9
problem with zone V5V 5E9
V5X 4K1
problem with zone V5X 4K1
V6A 1E3
problem with zone V6A 1E3
V3M 2M9
problem with zone V3M 2M9


'{"available": {"V6M 1X5": {"cars": [10.0, 9.0, 5.0, 9.0, 3.0, 1.0, 1.0, 2.0, 3.0, 5.0, 15.0, 20.0, 21.0, 6.0, 10.0, 6.0, 7.0, 9.0, 7.0, 11.0, 12.0, 12.0, 11.0, 11.0]}, "V5Y 3X2": {"cars": [10.0, 9.0, 5.0, 9.0, 3.0, 1.0, 1.0, 2.0, 3.0, 5.0, 15.0, 20.0, 21.0, 6.0, 10.0, 6.0, 7.0, 9.0, 7.0, 11.0, 12.0, 12.0, 11.0, 11.0]}, "V6M 1X1": {"cars": [10.0, 9.0, 5.0, 9.0, 3.0, 1.0, 1.0, 2.0, 3.0, 5.0, 15.0, 20.0, 21.0, 6.0, 10.0, 6.0, 7.0, 9.0, 7.0, 11.0, 12.0, 12.0, 11.0, 11.0]}, "V3M 3N7": {"cars": [10.0, 9.0, 5.0, 9.0, 3.0, 1.0, 1.0, 2.0, 3.0, 5.0, 15.0, 20.0, 21.0, 6.0, 10.0, 6.0, 7.0, 9.0, 7.0, 11.0, 12.0, 12.0, 11.0, 11.0]}, "V6A 3K1": {"cars": [10.0, 9.0, 5.0, 9.0, 3.0, 1.0, 1.0, 2.0, 3.0, 5.0, 15.0, 20.0, 21.0, 6.0, 10.0, 6.0, 7.0, 9.0, 7.0, 11.0, 12.0, 12.0, 11.0, 11.0]}, "V6C 1S4": {"cars": [10.0, 9.0, 5.0, 9.0, 3.0, 1.0, 1.0, 2.0, 3.0, 5.0, 15.0, 20.0, 21.0, 6.0, 10.0, 6.0, 7.0, 9.0, 7.0, 11.0, 12.0, 12.0, 11.0, 11.0]}, "V6G 3B1": {"cars": [10.0, 9.0, 5.0, 9.0, 3.0, 1.0, 1.0, 2.0, 3.0,

In [73]:
import re
a=re.findall('[a-zA-Z]\d[a-zA-Z]\s{0,1}\d{0,1}[a-zA-Z]{0,1}\d{0,1}','/opt/apps/Jaarvis_Demand_Supply/demand_models/V1M 2W4.pkl')[0]
a

'V1M 2W4'

In [77]:
Sql_connection2 = create_connection('/opt/apps/scripts/jaarvis_demand_supply/zone_lat_log.db')

In [81]:
lat_long_df = pd.read_sql_query("select * from zone_lat_log;", Sql_connection2)
lat_long_df

zone        lat         log
0      14340  34.184386 -118.445851
1      14430  34.148527 -118.447572
2      14640  47.632604 -122.143808
3    V3L 1M6  49.209934 -122.912854
4    V3L 1P2  49.211574 -122.914376
5    V3L 1P3  49.210620 -122.916248
6    V3L 1T8  49.214772 -122.916844
7    V3L 1V3  49.213286 -122.919269
8    V3L 1V9  49.219355 -122.913324
9    V3L 1W6  49.215750 -122.920154
10   V3L 1X7  49.220534 -122.915178
11   V3L 2E2  49.220273 -122.914056
12   V3L 2H3  49.218242 -122.912647
13   V3L 2J4  49.222821 -122.919109
14   V3L 2L4  49.213981 -122.911285
15   V3L 2M2  49.216316 -122.913404
16   V3L 2V4  49.212351 -122.915548
17   V3L 2W1  49.215782 -122.918947
18   V3L 3G9  49.223210 -122.906819
19   V3L 3Z7  49.228988 -122.895963
20   V3L 4A7  49.235013 -122.895416
21   V3L 4C8  49.233685 -122.898596
22   V3L 4G6  49.220909 -122.907085
23   V3L 4J1  49.224859 -122.905543
24   V3L 4K1  49.226226 -122.905148
25   V3L 4K4  49.227376 -122.903218
26   V3L 4M1  49.225993 -122.912801
27   V3L 4N5  49.231533 -122.902414
28   V3L 4N7  49.227145 -122.907677
29   V3L 4R5  49.226774 -122.912576
..       ...        ...         ...
894  V7P 2M3  49.328977 -123.099136
895  V7P 3C4  49.324003 -123.102155
896  V7P 3J6  49.320444 -123.104370
897  V7P 3N4  49.318579 -123.100499
898  V7P 3N9  49.324339 -123.096906
899  V7P 3P3  49.321423 -123.104092
900  V7P 3P7  49.319701 -123.099779
901  V7P 3P9  49.319833 -123.098540
902  V7P 3R8  49.316583 -123.097866
903  V7P 3T8  49.314821 -123.098368
904  V7R 4K9  49.375186 -123.091565
905  V7R 4T8  49.366939 -123.096723
906  V7R 4W1  49.369222 -123.098477
907  V7Y 0A1  49.283235 -123.117764
908  V7Y 1K8  49.282374 -123.119608
909  V6K 4R5  49.260333 -123.172042
910  V5P 0B8  49.222518 -123.071124
911  V5W 1Y5  49.228946 -123.083268
912  V6S 2L2  49.256128 -123.233788
913  V5M 3W5  49.268196 -123.031094
914  V5V 3Y1  49.252350 -123.096873
915  V6R 4M7  49.261552 -123.191085
916  V5R 5T9  49.248534 -123.031232
917  V5V 5E4  49.247467 -123.095766
918  V3L 3H3  49.226819 -122.911464
919  V5S 3H2  49.225271 -123.036794
920  V6N 1P1  49.237889 -123.178394
921  V3L 3Z5  49.228277 -122.895969
922  V6N 1B5  49.241118 -123.165794
923  V5M 0G4  49.255220 -123.055050

[924 rows x 3 columns]

In [79]:
cursor = Sql_connection2.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('zone_lat_log',)]


In [108]:
zone='V6B 2J2'
lat_long_df[lat_long_df['zone'].str.contains(zone)]['lat'].values[0],lat_long_df[lat_long_df['zone'].str.contains(zone)]['log'].values[0]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [2]:
import re
import sqlite3
import numpy
import pandas as pd
import os
from sqlite3 import Error
import glob
import pickle
import json
from collections import defaultdict

class forecasting():

    def __init__(self):

        self.zones_list = pd.read_csv('opt/apps/Jaarvis_Demand_Supply/zones_list.csv')
        self.Sql_connection1 = self.create_connection('/opt/apps/scripts/jaarvis_demand_supply/zone_lat_log.db')
        self.lat_long_df = pd.read_sql_query("select * from zone_lat_log;", self.Sql_connection1)
        self.path1 = '/opt/apps/Jaarvis_Demand_Supply/demand_models'
        self.path2 = '/opt/apps/Jaarvis_Demand_Supply/available_models'

    def create_connection(self, db_file):
        """ create a database connection to the SQLite database
            specified by the db_file
        :param db_file: database file
        :return: Connection object or None
        """
        try:
            conn = sqlite3.connect(db_file)
            return conn
        except Error as e:
            print(e)
        print('DB connection status checking')
        return None

    def forecaster(self):

        booked = defaultdict(dict)
        available = defaultdict(dict)
        demand = defaultdict(dict)
        for x in glob.glob(os.path.join(self.path1, '*.pkl')):
            try:
                model_book = pickle.load(open(x, 'rb'))
                zone = re.findall('[a-zA-Z]\d[a-zA-Z]\s{0,1}\d{0,1}[a-zA-Z]{0,1}\d{0,1}', x)[0]
                # print(zone)
                demand[zone]['booked'] = [0 if i < 0 else i for i in numpy.ceil((model_book.forecast(24)))]
                # print(self.lat_long_df[self.lat_long_df['zone'].str.contains(zone)]['lat'].values[0],
                #       self.lat_long_df[self.lat_long_df['zone'].str.contains(zone)]['log'].values[0])
                demand[zone]['lat_long'] = self.lat_long_df[self.lat_long_df['zone'].str.contains(zone)]['lat'].values[0], \
                                           self.lat_long_df[self.lat_long_df['zone'].str.contains(zone)]['log'].values[0]
            except:
                # print('problem with zone ' + zone)
                pass
        for y in glob.glob(os.path.join(self.path2, '*.pkl')):
            try:
                model_free = pickle.load(open(y, 'rb'))
                zone = re.findall('[a-zA-Z]\d[a-zA-Z]\s{0,1}\d{0,1}[a-zA-Z]{0,1}\d{0,1}', y)[0]
                # print(zone)
                demand[zone]['available'] = [0 if i < 0 else i for i in numpy.ceil((model_free.forecast(24)))]
                # print(self.lat_long_df[self.lat_long_df['zone'].str.contains(zone)]['lat'].values[0],
                #       self.lat_long_df[self.lat_long_df['zone'].str.contains(zone)]['log'].values[0])
                demand[zone]['lat_long'] = self.lat_long_df[self.lat_long_df['zone'].str.contains(zone)]['lat'].values[0], \
                                           self.lat_long_df[self.lat_long_df['zone'].str.contains(zone)]['log'].values[0]

            except:
                # print('problem with zone ' + zone)
                pass
        # demand['booked'] = booked
        # demand['available'] = available
        data = json.dumps(demand)
        with open('demand.json', 'w') as f:
            f.write(data)

forecastingObj = forecasting()
forecastingObj.forecaster()

In [8]:
pd.read_json('demand.json')

V1M 2R6  \
available  [-0.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0...   
booked     [-0.0, 1.0, -0.0, -0.0, 1.0, 1.0, 1.0, -0.0, -...   
lat_long                           [49.165906, -122.5705149]   

                                                     V1M 3G1  \
available  [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0....   
booked     [-0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...   
lat_long                          [49.1810513, -122.6609925]   

                                                     V2J 3A9  \
available  [-0.0, 1.0, -0.0, 1.0, -0.0, 1.0, -0.0, 1.0, -...   
booked     [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0,...   
lat_long                          [52.9752637, -122.4735091]   

                                                     V3C 2S9  \
available  [1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...   
booked     [-0.0, -0.0, 1.0, 1.0, -0.0, -0.0, -0.0, -0.0,...   
lat_long                          [49.2448805, -122.7759246]   

                                                     V3J 3Z9  \
available  [-0.0, 1.0, 1.0, -0.0, 1.0, -0.0, 1.0, 1.0, 1....   
booked     [-0.0, 1.0, 1.0, -0.0, 1.0, -0.0, 1.0, 1.0, 1....   
lat_long                          [49.2686959, -122.8868533]   

                                                     V3L 0C9  \
available  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...   
booked                                                   NaN   
lat_long                   [49.20871469999999, -122.9150029]   

                                                     V3L 0E3  \
available  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...   
booked     [-0.0, 1.0, -0.0, 1.0, -0.0, 1.0, 1.0, 1.0, -0...   
lat_long                          [49.2241052, -122.8923865]   

                                                     V3L 0E7  \
available  [1.0, -0.0, 1.0, -0.0, 1.0, -0.0, 1.0, -0.0, 1...   
booked     [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0....   
lat_long                            [49.216698, -122.900781]   

                                                     V3L 0H4  \
available  [-0.0, -0.0, -0.0, -0.0, 1.0, 1.0, 1.0, -0.0, ...   
booked     [-0.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0...   
lat_long                          [49.2209898, -122.8990775]   

                                                     V3L 1A7  \
available  [3.0, 1.0, -0.0, 1.0, -0.0, 1.0, -0.0, 1.0, -0...   
booked     [2.0, 1.0, 1.0, -0.0, 1.0, -0.0, -0.0, -0.0, -...   
lat_long                           [49.205517, -122.9039549]   

                                 ...                          \
available                        ...                           
booked                           ...                           
lat_long                         ...                           

                                                     V7N 1X1  \
available  [-0.0, 1.0, -0.0, 1.0, -0.0, -0.0, 1.0, -0.0, ...   
booked     [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...   
lat_long                          [49.3363104, -123.0671135]   

                                                         V7P  \
available  [2.0, 2.0, -0.0, -0.0, 1.0, 1.0, -0.0, 1.0, -0...   
booked     [5.0, 2.0, 1.0, 2.0, -0.0, -0.0, -0.0, 1.0, 1....   
lat_long                          [49.3237588, -123.0982218]   

                                                     V7P 1E4  \
available                                                NaN   
booked     [1.0, -0.0, 1.0, -0.0, 1.0, -0.0, 1.0, -0.0, 1...   
lat_long                   [49.32001349999999, -123.1012908]   

                                                     V7P 1S2  \
available  [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0....   
booked     [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0....   
lat_long                           [49.322799, -123.1000967]   

                                                     V7R 4L9  \
available  [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0....   
booked     [-0.0, 1.0, -0.0, 1.0, -0.0, 1.0, -0.0, 1.0, -...   
lat_lo

In [38]:
#!/usr/bin/env python
# coding: utf-8

'''demand_train.py creates SARIMA models for all the zones we want to predict demand for'''

import numpy
import datetime
import sqlite3
from sqlite3 import Error
from sklearn import preprocessing
import pandas as pd
import pickle
import statsmodels.api
import glob
import pickle
import json
from collections import defaultdict
import os


class DemandTrain:
    def __init__(self):
        # try:
        #     if platform.system() == 'Windows':
        self.Sql_connection1 = self.create_connection('/opt/apps/scripts/jaarvis_demand_supply/evo.db')

        #     elif platform.system() == 'Linux':
        #         self.Sql_connection1 = self.create_connection('/opt/apps/scripts/jaarvis_demand_supply/evo.db')
        # except Error as e:
        #     print(e)

        self.df = pd.read_sql_query("select * from zones order by id;", self.Sql_connection1)

    def create_connection(self, db_file):
        """ create a database connection to the SQLite database
            specified by the db_file
        :param db_file: database file
        :return: Connection object or None
        """
        try:
            conn = sqlite3.connect(db_file)
            return conn
        except Error as e:
            print(e)
        print('DB connection is done')
        return None

    def data_cleansing(self):
        print('data_cleansing started')
        self.df.booked_vehicles = self.df.booked_vehicles.fillna(0)
        self.df.free_vehicles = self.df.free_vehicles.fillna(0)
        self.df = self.df.dropna(how='any')
        self.df = self.df[~self.df.astype(str).eq('None').any(1)]
        self.df = self.df.mask(self.df.eq('None')).dropna()
        self.df = self.df[~self.df.zone.apply(lambda x: str(x).isdigit())]
        self.df.time = self.df.time.astype(str).str[:2].astype(int)
        self.df.time = self.df.time.replace(regex=True, to_replace=r'\D', value=r'')
        self.df = self.df.sort_values(['zone', 'year', 'month', 'week', 'day', 'time'], ascending=True)
        self.df['date'] = self.df[['year', 'month', 'day']].apply(lambda x: '-'.join(x.dropna().astype(str).values),
                                                                    axis=1)
        self.df = self.df.drop(['id', 'year', 'month', 'week', 'day'], axis=1)
        self.df['date'] = self.df[['date', 'time']].apply(lambda x: ' '.join(x.dropna().astype(str).values), axis=1)
        self.df['date'] = self.df[['date', 'time']].apply(lambda x: x.astype(str) + ':00:00', axis=1)
        self.df = self.df.sort_values(['date', 'zone'])
        print('data cleansing finished')
        return self.df

    def feature_addition(self):
        # no feature added
        print('feature addition started')
        train_df = self.data_cleansing()
        train_booked_df = pd.pivot_table(train_df, values=['booked_vehicles'], index=['date'], columns=['zone'])
        train_free_df = pd.pivot_table(train_df, values=['free_vehicles'], index=['date'], columns=['zone'])
        train_booked_df = train_booked_df.fillna(0)
        train_free_df = train_free_df.fillna(0)
        train_booked_df = train_booked_df.booked_vehicles
        train_free_df = train_free_df.free_vehicles
        booked_df = train_booked_df.reindex(
            pd.date_range(start='2018-11-16 14:00:00', end=train_booked_df.index[-1], freq='H'))
        free_df = train_free_df.reindex(
            pd.date_range(start='2018-11-16 14:00:00', end=train_free_df.index[-1], freq='H'))
        booked_df = booked_df[~booked_df.index.isin(train_booked_df.index)]
        free_df = free_df[~free_df.index.isin(train_free_df.index)]
        train_booked_df = pd.concat([train_booked_df, booked_df])
        train_free_df = pd.concat([train_free_df, free_df])
        print('features added')
        return train_booked_df, train_free_df

    def feature_elimination(self):
        # no feature deleted
        pass

    def training_data(self):
        print('making training data')
        train_booked_df, train_free_df = self.feature_addition()
        train_booked_df = train_booked_df.iloc[pd.to_datetime(train_booked_df.index).values.argsort()]
        train_free_df = train_free_df.iloc[pd.to_datetime(train_free_df.index).values.argsort()]
        train_booked_df = train_booked_df.fillna(0)
        train_free_df = train_free_df.fillna(0)
        train_booked_df.index = pd.to_datetime(train_booked_df.index) - pd.Timedelta(hours=5)
        train_free_df.index = pd.to_datetime(train_free_df.index) - pd.Timedelta(hours=5)

        pd.DataFrame({'zone': self.df['zone'].unique()}).to_csv("zones_list.csv")
        print('training data ready for modeling')
        return train_booked_df, train_free_df

    def train(self):
        train_booked_df, train_free_df = self.training_data()
        path1 = '/opt/apps/Jaarvis_Demand_Supply/demand_models/'
        path2 = '/opt/apps/jaarvis_Demand_Supply/available_models/'
        print('training started')
        os.chdir(path1)
        for x in train_booked_df.keys():
            try:

                model = statsmodels.api.tsa.SARIMAX(train_booked_df[x][-171:-171 + 168], order=(1, 0, 1),
                                                    seasonal_order=(1, 1, 0, 24), enforce_stationarity=True)
                model = model.fit()
                model = model.save(str(x) + '.pkl')
            except:

                print(x, 'model not created', )

        print('demand_models_created')

        os.chdir(path2)
        for y in train_free_df.keys():
            try:

                model = statsmodels.api.tsa.SARIMAX(train_free_df[y][-171:-171 + 168], order=(1, 0, 1),
                                                    seasonal_order=(1, 1, 0, 24), enforce_stationarity=True)
                model = model.fit()
                model = model.save(str(y) + '.pkl')
            except:
                print(y, 'model not created')
        print('available_models_created\ntraining finished')


TrainingObj = DemandTrain()
TrainingObj.train()

making training data
feature addition started
data_cleansing started
data cleansing finished
features added
training data ready for modeling
training started
V1M 2R6 model not created
V1M 3G1 model not created
V2J 3A9 model not created


/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packa

V3J 3Z9 model not created
V3K 7B3 model not created
V3L 0A4 model not created
V3L 0A5 model not created


/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packa

V3L 1G6 model not created
V3L 1J6 model not created
V3L 1J7 model not created


...

V7R 4K9 model not created
V7R 4L9 model not created
V7R 4N4 model not created
V7R 4P6 model not created
V7R 4T8 model not created
V7R 4V1 model not created
V7R 4V3 model not created
V7R 4V6 model not created
V7R 4W1 model not created
V7R 4W6 model not created


/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)


V7X 1M7 model not created
V7Y model not created
V7Y 1A1 model not created
V7Y 1G5 model not created
demand_models_created


/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)


FileNotFoundError: [Errno 2] No such file or directory: '/opt/apps/jaarvis_Demand_Supply/available_models/'

In [ ]:
import re
import sqlite3
import numpy
import pandas as pd
import os
from sqlite3 import Error
import glob
import pickle
import json
from collections import defaultdict

class forecasting():

    def __init__(self):

        self.zones_list = pd.read_csv('/opt/apps/Jaarvis_Demand_Supply/zones_list.csv')
        self.Sql_connection1 = self.create_connection('/opt/apps/scripts/jaarvis_demand_supply/zone_lat_log.db')
        self.lat_long_df = pd.read_sql_query("select * from zone_lat_log;", self.Sql_connection1)
        self.path1 = '/opt/apps/Jaarvis_Demand_Supply/demand_models'
        self.path2 = '/opt/apps/Jaarvis_Demand_Supply/available_models'

    def create_connection(self, db_file):
        """ create a database connection to the SQLite database
            specified by the db_file
        :param db_file: database file
        :return: Connection object or None
        """
        try:
            conn = sqlite3.connect(db_file)
            return conn
        except Error as e:
            print(e)
        return None
        print("DB connection check..")

    def forecaster(self):
        print("forecaster method is reading the data now")
        booked = defaultdict(dict)
        available = defaultdict(dict)
        demand = defaultdict(dict)
        for x in glob.glob(os.path.join(self.path1, '*.pkl')):
            try:
                model_book = pickle.load(open(x, 'rb'))
                zone = re.findall('[a-zA-Z]\d[a-zA-Z]\s{0,1}\d{0,1}[a-zA-Z]{0,1}\d{0,1}', x)[0]
                print(zone)
                demand[zone]['booked'] = [0 if i < 0 else i for i in numpy.ceil((model_book.forecast(24)))]
                # print(self.lat_long_df[self.lat_long_df['zone'].str.contains(zone)]['lat'].values[0],
                #       self.lat_long_df[self.lat_long_df['zone'].str.contains(zone)]['log'].values[0])
                demand[zone]['lat_long'] = self.lat_long_df[self.lat_long_df['zone'].str.contains(zone)]['lat'].values[0], \
                                           self.lat_long_df[self.lat_long_df['zone'].str.contains(zone)]['log'].values[0]
            except:
                print('problem with zone ' + zone)
                pass
        for y in glob.glob(os.path.join(self.path2, '*.pkl')):
            try:
                model_free = pickle.load(open(y, 'rb'))
                zone = re.findall('[a-zA-Z]\d[a-zA-Z]\s{0,1}\d{0,1}[a-zA-Z]{0,1}\d{0,1}', y)[0]
                print(zone)
                demand[zone]['available'] = [0 if i < 0 else i for i in numpy.ceil((model_free.forecast(24)))]
                # print(self.lat_long_df[self.lat_long_df['zone'].str.contains(zone)]['lat'].values[0],
                #       self.lat_long_df[self.lat_long_df['zone'].str.contains(zone)]['log'].values[0])
                demand[zone]['lat_long'] = self.lat_long_df[self.lat_long_df['zone'].str.contains(zone)]['lat'].values[0], \
                                           self.lat_long_df[self.lat_long_df['zone'].str.contains(zone)]['log'].values[0]

            except:
                print('problem with zone ' + zone)
                pass
        # demand['booked'] = booked
        # demand['available'] = available
        data = json.dumps(demand)
        with open('demand.json', 'w') as f:
            f.write(data)
        print('forecasting done json saved in demand.json')

forecastingObj = forecasting()
forecastingObj.forecaster()

In [36]:
path1 = '/opt/apps/Jaarvis_Demand_Supply/demand_models/'
path2 = '/opt/apps/Jaarvis_Demand_Supply/available_models/'
# print('training started')
# os.chdir(path1)
# for x in train_booked_df.keys():
#     try:

#         model = statsmodels.api.tsa.SARIMAX(train_booked_df[x][-171:-171 + 168], order=(1, 0, 1),
#                                             seasonal_order=(1, 1, 0, 24), enforce_stationarity=False)
#         model = model.fit()
#         model = model.save(str(x) + '.pkl')
#     except:

#         print(x, 'model not created', )

# print('demand_models_created')

os.chdir(path2)
for y in train_free_df.keys():
    try:

        model = statsmodels.api.tsa.SARIMAX(train_free_df[y][-171:-171 + 168], order=(1, 0, 1),
                                            seasonal_order=(1, 1, 0, 24), enforce_stationarity=False)
        model = model.fit()
        model = model.save(str(y) + '.pkl')
    except:
        print(y, 'model not created')
print('available_models_created\ntraining finished')


/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)


V1M 2R6 model not created
V1M 3G1 model not created
V2J 3A9 model not created


/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)


V3J 3Z9 model not created
V3K 7B3 model not created
V3L 0A4 model not created
V3L 0A5 model not created


/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packa

...

/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)


V7X 1M7 model not created
V7Y model not created
V7Y 1A1 model not created


/usr/local/lib/python3.5/dist-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


V7Y 1G5 model not created
available_models_created
training finished


/usr/local/lib/python3.5/dist-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
